In [ ]:
feature_model = torch.load(root_dir + 'FeatureNet.pkl')
print (feature_model)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data
import torch.nn.init as init
import pytorch_ssim

import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import pickle
import os
import json
import numpy as np
import gc
import cv2

In [ ]:
epoch = 5
lr = 0.001
mom = 0.9
bs = 10

In [ ]:
def odd(w):
    return list(np.arange(1, w, step=2, dtype='long'))

def even(w):
    return list(np.arange(0, w, step=2, dtype='long'))

def white(x):
    fw, tw = x.shape[1], x.shape[2]

    first = F.relu(torch.normal(mean=torch.zeros(fw, tw), std=torch.ones(fw, tw)) ) * 0.05
    second_seed = F.relu(torch.normal(mean=torch.zeros(fw//2, tw//2), std=torch.ones(fw//2, tw//2))) * 0.03
    second = torch.zeros(fw, tw)

    row_x  = torch.zeros(int(fw//2), tw)
    # row_x = torch.zeros(int(fw/2), tw)

    row_x[:, odd(tw)]  = second_seed
    row_x[:, even(tw)] = second_seed

    second[odd(fw), :]  = row_x
    second[even(fw), :] = row_x

    return second + first


In [ ]:
#=============================================
#        path
#=============================================

server = False

root_dir = '/home/tk/Documents/'
if server == True:
    root_dir = '/home/guotingyou/cocktail_phase2/'


clean_dir = root_dir + 'clean/' 
# mix_dir = root_dir + 'mix/' 
# clean_label_dir = root_dir + 'clean_labels/' 
# mix_label_dir = root_dir + 'mix_labels/' 

cleanfolder = os.listdir(clean_dir)
cleanfolder.sort()

# mixfolder = os.listdir(mix_dir)
# mixfolder.sort()


clean_list = []
# mix_list = []

In [ ]:
class MSourceDataSet(Dataset):
    
    def __init__(self, clean_dir):
        

        # Overfitting single block
        with open(clean_dir + 'clean11.json') as f:
            clean_list.append(torch.Tensor(json.load(f)))

            
        cleanblock = torch.cat(clean_list, 0)
#         mixblock = torch.cat(mix_list, 0)
        self.spec = cleanblock
                
        
    def __len__(self):
        return self.spec.shape[0]

                
    def __getitem__(self, index): 

        spec = self.spec[index]
        return spec
    
#=============================================
#        Define Dataloader
#=============================================

trainset = MSourceDataSet(clean_dir)

trainloader = torch.utils.data.DataLoader(dataset = trainset,
                                                batch_size = bs,
                                                shuffle = True)
#=============================================
#        Define Dataloader
#=============================================

trainset = MSourceDataSet(clean_dir)

trainloader = torch.utils.data.DataLoader(dataset = trainset,
                                                batch_size = bs,
                                                shuffle = True)

In [ ]:
#=============================================
#        Model
#=============================================

''' ResBlock '''
class ResBlock(nn.Module):
    def __init__(self, channels_in, channels_out):
        super(ResBlock, self).__init__()

        self.channels_in = channels_in
        self.channels_out = channels_out

        self.conv1 = nn.Conv2d(in_channels=channels_in, out_channels=channels_out, kernel_size=(3,3), padding=1)
        self.conv2 = nn.Conv2d(in_channels=channels_out, out_channels=channels_out, kernel_size=(3,3), padding=1)

    def forward(self, x):
        if self.channels_out > self.channels_in:
            x1 = F.relu(self.conv1(x), inplace = True)
            x1 =        self.conv2(x1)
            x  = self.sizematch(self.channels_in, self.channels_out, x)
            return F.relu(x + x1, inplace = True)
        elif self.channels_out < self.channels_in:
            x = F.relu(self.conv1(x))
            x1 =       self.conv2(x)
            x = x + x1
            return F.relu(x, inplace = True)
        else:
            x1 = F.relu(self.conv1(x), inplace = True)
            x1 =        self.conv2(x1)
            x = x + x1
            return F.relu(x, inplace = True)

    def sizematch(self, channels_in, channels_out, x):
        zeros = torch.zeros( (x.size()[0], channels_out - channels_in, x.shape[2], x.shape[3]), dtype = torch.float32)
        return torch.cat((x, zeros), dim=1)

class ResTranspose(nn.Module):
    def __init__(self, channels_in, channels_out):
        super(ResTranspose, self).__init__()

        self.channels_in = channels_in
        self.channels_out = channels_out

        self.deconv1 = nn.ConvTranspose2d(in_channels=channels_in, out_channels=channels_out, kernel_size=(2,2), stride=2)
        self.deconv2 = nn.Conv2d(in_channels=channels_out, out_channels=channels_out, kernel_size=(3,3), padding=1)

    def forward(self, x):
        # cin = cout
        x1 = F.relu(self.deconv1(x), inplace = True)
        x1 =        self.deconv2(x1)
        x = self.sizematch(x)
        return F.relu(x + x1, inplace = True)

    def sizematch(self, x):
        # expand
        x2 = torch.zeros(x.shape[0], self.channels_in, x.shape[2]*2, x.shape[3]*2)

        row_x  = torch.zeros(x.shape[0], self.channels_in, x.shape[2], 2*x.shape[3])
        row_x[:,:,:,odd(x.shape[3]*2)]   = x
        row_x[:,:,:,even(x.shape[3]*2)]  = x
        x2[:,:, odd(x.shape[2]*2),:] = row_x
        x2[:,:,even(x.shape[2]*2),:] = row_x

        return x2


def initialize(m):
    if isinstance(m, nn.Conv2d):
        init.xavier_normal_(m.weight)
        init.constant_(m.bias, 0)
    if isinstance(m, nn.ConvTranspose2d):
        init.xavier_normal_(m.weight)





# model = ResDAE()
model = torch.load(root_dir + 'recover/SSIM-CONV/DAE_SSIM.pkl')
print (model)


In [ ]:

#=============================================
#        Optimizer
#=============================================

#import pytorch_ssim
criterion = pytorch_ssim.SSIM()
optimizer = torch.optim.Adam(model.parameters(), lr = lr) #, momentum = mom)

#=============================================
#        Loss Record
#=============================================

loss_record = []
# every_loss = []
# epoch_loss = []

#=============================================
#        test
#=============================================

criterion = pytorch_ssim.SSIM()

model.eval()
for epo in range(epoch):
    for i, data in enumerate(trainloader, 0):
        inputs = data
        inputs = Variable(inputs)
        top = model.upward(inputs + white(inputs))
        
        outputs = model.downward(top, shortcut = True)
        inputs = inputs.view(bs, 1, 256, 128)
        outputs = outputs.view(bs, 1, 256, 128)
        #with open ( root_dir + 'recover/L1loss_FC/recover_pic_epo_' + str(epo), 'w') as f:
        #    json.dump(outputs.tolist(), f)
        
        loss = - criterion(outputs, inputs)
        ssim_value = - loss.data.item()
        
        if i % 20 == 0:
            inn = inputs[0].view(256, 128).detach().numpy() * 255
            cv2.imwrite("/home/tk/Documents/recover/SSIM-CONV/test/" + str(epo) + "_" + str(i) + ".png", inn)
            
            out = outputs[0].view(256, 128).detach().numpy() * 255
            cv2.imwrite("/home/tk/Documents/recover/SSIM-CONV/test/" + str(epo) + "_" + str(i) + "_re.png", out)
            

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset

import matplotlib.pyplot as plt
import pickle
import os
import json
import numpy as np



#=============================================
#        Hyperparameters
#=============================================

epoch = 2
lr = 0.001
mom = 0.9
bs = 16

#======================================
clean_dir = '/home/tk/Documents/clean/' 
clean_label_dir = '/home/tk/Documents/clean_labels/' 
#========================================

cleanfolder = os.listdir(clean_dir)
cleanfolder.sort()

cleanlabelfolder = os.listdir(clean_label_dir)
cleanlabelfolder.sort()

clean_list = []
clean_label_list = []

#========================================

class featureDataSet(Dataset):
    
    def __init__(self, clean_dir, clean_label_dir):
                

        for i in cleanfolder:
            with open(clean_dir + '{}'.format(i)) as f:
                clean_list.append(torch.Tensor(json.load(f)))
                
        for i in cleanlabelfolder:
            with open(clean_label_dir + '{}'.format(i)) as f:
                clean_label_list.append(torch.Tensor(json.load(f)))
        
        cleanblock = torch.cat(clean_list, 0)
        self.spec = torch.cat([cleanblock], 0)
                
        cleanlabel = torch.cat(clean_label_list, 0)
        self.label = torch.cat([cleanlabel], 0)

        
    def __len__(self):
        return self.spec.shape[0]

                
    def __getitem__(self, index): 

        spec = self.spec[index]
        label = self.label[index]
        return spec, label

    
#=================================================    
#           Dataloader 
#=================================================
featureset = featureDataSet(clean_dir, clean_label_dir)
trainloader = torch.utils.data.DataLoader(dataset = featureset,
                                                batch_size = bs,
                                                shuffle = True)

#=================================================    
#           model 
#=================================================
class featureNet(nn.Module):
    def __init__(self):
        super(featureNet, self).__init__()
        self.fc1 = nn.Linear(256 * 128, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 10)

        
    def forward(self, x):
        x = x.view(-1, 256*128)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        
        return x
    
#model = featureNet()
model = torch.load('/home/tk/Documents/FeatureNet.pkl')
print (model)

#============================================
#              optimizer
#============================================
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = lr, momentum = mom)

#============================================
#              training
#============================================

loss_record = []
every_loss = []
epoch_loss = []


model.train()
for epoch in range(70):
    
    
    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data

        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss_record.append(loss.item())
        every_loss.append(loss.item())
        print ('[%d, %5d] loss: %.3f' % (epoch, i, loss.item()))
        
    epoch_loss.append(np.mean(every_loss))
    every_loss = []

            
            
torch.save(model, '/home/tk/Documents/FeatureNet.pkl')


plt.figure(figsize = (20, 10))
plt.plot(loss_record)
plt.xlabel('iterations')
plt.ylabel('loss')
plt.savefig('loss.png')
plt.show()

plt.figure(figsize = (20, 10))
plt.plot(epoch_loss)
plt.xlabel('iterations')
plt.ylabel('epoch_loss')
plt.savefig('epoch_loss')
plt.show()

In [ ]:
model = torch.load('/home/tk/Documents/FeatureNet.pkl')
print (model)

In [ ]:
j = open('/home/tk/Documents/mix/mix0.json').read()
a = json.loads(j)
mix = np.array(a)
print (mix.shape)

In [ ]:
j = open('/home/tk/Documents/mix_labels/mix_label0.json').read()
a = json.loads(j)
mix_label = np.array(a)
print(mix_label.shape)

In [ ]:
mix_label

In [ ]:
i = 2
index1 = (i * 11) + mix_label[i][0]
index2 = (i * 11) + mix_label[i][1]
print (index1)
print (index2)
mix_index = i * 11 + 10
print (mix_index)

In [ ]:
i = 2
index1 = (i * 11) + mix_label[i][0]
index2 = (i * 11) + mix_label[i][1]
print (index1)
print (index2)
mix_index = i * 11 + 10
print (mix_index)

In [ ]:
mix_label[2][1]

In [ ]:
cv2.imwrite('/home/tk/Desktop/a.png', mix[index1] * 255)
print (mix[index1])

In [ ]:
cv2.imwrite('/home/tk/Desktop/b.png', mix[index2] * 255)
print (mix[index2])

In [ ]:
print (mix[mix_index])

In [ ]:
c = (mix[index1] + mix[index2])
c = np.clip(c, np.min(c), 1)

In [ ]:
np.max(c)

In [ ]:
cv2.imwrite('/home/tk/Desktop/mix.png', c * 255)
print (mix[mix_index])

In [ ]:
print (mix_label[3])

In [ ]:
3 * 11 + 5

In [ ]:
print (mix_label[3][0])

In [ ]:
with open('/home/tk/Documents/clean/clean0.json') as f:
    b = f.read()

In [ ]:
j = open('/home/tk/Documents/clean/clean0.json').read()
b = json.loads(b)
print (b)

In [ ]:
with open('/home/tk/Documents/mix_labels/mix_label0.json') as f:
    a = f.read()

In [ ]:
print (len(a))

In [ ]:
b = [1,2,3,4,5,6]

In [ ]:
z = np.zeros((10))
print (z[0])

In [ ]:
root_dir = '/home/tk/Documents/'
clean_dir = root_dir + 'clean/' 

clean_list = []
clean_label_list = []
mix_list = []
mix_label_list = []
import torch

In [ ]:
i = 0
c[i*2: (i+1)*2 ]

In [ ]:
c = [[1],[1],[2],[2]]

In [ ]:
class featureDataSet(Dataset):
    
    def __init__(self, c6lean_dir):

        with open(clean_dir + 'clean3.json') as f:
            clean_list.append(torch.Tensor(json.load(f)))
        
        with open(root_dir + "clean_labels/clean_label3.json") as f:
            clean_label_list.append(torch.Tensor(json.load(f)))

        with open(root_dir + 'mix/mix0.json') as f:
            mix_list.append(torch.Tensor(json.load(f)))
            
        with open(root_dir + 'mix_labels/mix_label0.json') as f:
            mix_label_list.append(torch.Tensor(json.load(f)))
            
        cleanblock = torch.cat(clean_list, 0)
        mixblock = torch.cat(mix_list, 0)
        self.spec = cleanblock
                
        
    def __len__(self):
        return self.spec.shape[0]

                
    def __getitem__(self, index): 

        spec = self.spec[index]
        return spec


In [ ]:
for h in range(0, 200, 20):
    print (h)

In [ ]:
length = 0.5
multiplication = 1

In [ ]:
spec_name = os.listdir(root_dir + "slice_pointsec/")
if len(spec_name) == (10/length)* 10 * multiplication: 
    print ((10/length)* 10 * multiplication)
    print ("checked")
spec_name.sort()
print (spec_name[19])

In [ ]:
spec = data_process.gen_spectrogram(root_dir + 'slice_pointcec/' + spec_name[])

In [ ]:
for j in range(0 + i, int(10/length) * 10 * multiplication + i, int(10/length) * multiplication):
    spec = data_process.gen_spectrogram(root_dir + 'slice_pointsec/' + spec_name[j])
    small_pcs.append(spec)

In [ ]:
for j in range(0, 200, 20):
    print (j)

In [ ]:
json.close()

In [ ]:
l = [1,2,3,4,5,6,7]
np.clip(l, min(l), 5)

In [ ]:
server = False

root_dir = '/home/tk/Documents/'
if server == True:
    root_dir = '/home/guotingyou/cocktail_phase2/'


clean_dir = root_dir + 'clean/' 
mix_dir = root_dir + 'mix/' 
clean_label_dir = root_dir + 'clean_labels/' 
mix_label_dir = root_dir + 'mix_labels/' 

cleanfolder = os.listdir(clean_dir)
cleanfolder.sort()

mixfolder = os.listdir(mix_dir)
mixfolder.sort()


clean_list = []
mix_list = []
mix_label_list = []
feature_list = []

In [ ]:
class mixDataSet(Dataset):
    
    def __init__(self, mix_dir, mix_label_dir):           
        
        with open(mix_dir + 'mix3.json') as f:
            mix_list.append(torch.Tensor(json.load(f)))
        
        with open(mix_label_dir + 'mix_label3.json') as f:
            mix_label_list.append(torch.Tensor(json.load(f)))
        
        mixblock = torch.cat(mix_list, 0)
        mixlabel = torch.cat(mix_label_list, 0)
        
        self.mix_spec = mixblock
        self.mix_label = mixlabel
                
        
    def __len__(self):
        return self.mix_spec.shape[0]

                
    def __getitem__(self, index): 

        mix_spec = self.mix_spec[index]
        mix_label = self.mix_label[index]
        return mix_spec, mix_label


class featureDataSet(Dataset):
    
    def __init__(self, clean_dir):
        

        with open(clean_dir + 'clean3.json') as f:
            feature_list.append(torch.Tensor(json.load(f)))      
        
        featureblock = torch.cat(feature_list, 0)
        self.featurespec = featureblock
                
        
    def __len__(self):
        return self.featurespec.shape[0]

                
    def __getitem__(self, index): 

        featurespec = self.featurespec[index]
        return featurespec

In [ ]:
mixset = mixDataSet(mix_dir, mix_label_dir)

In [ ]:
mspec, mlabel = mixset.__getitem__(0)

In [ ]:
mlabel[0]

In [ ]:
featureset = featureDataSet(clean_dir)

In [ ]:
featureset.__getitem__()

In [ ]:
mixloader = torch.utils.data.DataLoader(dataset = MSourceDataSet(clean_dir),
                                                batch_size = bs,
                                                shuffle = False)

In [ ]:
mixset = mixDataSet(mix_dir, mix_label_dir)
featureset = featureDataSet(clean_dir)

mixloader = torch.utils.data.DataLoader(dataset = MSourceDataSet(clean_dir),
                                                batch_size = bs,
                                                shuffle = False)

featureloader = torch.utils.data.DataLoader(dataset = featureset,
                                           batch_size = bs,
                                           shuffle = False)


In [ ]:
mixset.__len__()

In [ ]:
mix_spec, mix_label = mixset.__getitem__(10)

In [ ]:
mix_spec

In [ ]:
mix_label[0]

In [ ]:
featureset.__getitem__()

In [ ]:
j = open(clean_dir + 'clean3.json').read()
b = json.loads(b)
print (b)

In [ ]:
j = open('/home/tk/Documents/clean/clean0.json').read()
a = json.loads(j)
mix = np.array(a)
print (mix.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data
import torch.nn.init as init
import pytorch_ssim

import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import pickle
import os
import json
import numpy as npu
import gc
import cv2

In [ ]:
epoch = 2
lr = 0.001
mom = 0.9
bs = 10

In [ ]:
def odd(w):
    return list(np.arange(1, w, step=2, dtype='long'))

def even(w):
    return list(np.arange(0, w, step=2, dtype='long'))

def white(x):
    fw, tw = x.shape[1], x.shape[2]

    first = F.relu(torch.normal(mean=torch.zeros(fw, tw), std=torch.ones(fw, tw)) ) * 0.05
    second_seed = F.relu(torch.normal(mean=torch.zeros(fw//2, tw//2), std=torch.ones(fw//2, tw//2))) * 0.03
    second = torch.zeros(fw, tw)

    row_x  = torch.zeros(int(fw//2), tw)
    # row_x = torch.zeros(int(fw/2), tw)

    row_x[:, odd(tw)]  = second_seed
    row_x[:, even(tw)] = second_seed

    second[odd(fw), :]  = row_x
    second[even(fw), :] = row_x

    return second + first

In [ ]:
server = False

root_dir = '/home/tk/Documents/'
if server == True:
    root_dir = '/home/guotingyou/cocktail_phase2/'


clean_dir = root_dir + 'clean/' 
mix_dir = root_dir + 'mix/' 
clean_label_dir = root_dir + 'clean_labels/' 
mix_label_dir = root_dir + 'mix_labels/' 

cleanfolder = os.listdir(clean_dir)
cleanfolder.sort()

mixfolder = os.listdir(mix_dir)
mixfolder.sort()


clean_list = []
mix_list = []
mix_label_list = []
feature_list = []

In [ ]:
class mixDataSet(Dataset):
    
    def __init__(self, mix_dir, mix_label_dir):           
        
        with open(mix_dir + 'mix3.json') as f:
            mix_list.append(torch.Tensor(json.load(f)))
        
        with open(mix_label_dir + 'mix_label3.json') as f:
            mix_label_list.append(torch.Tensor(json.load(f)))
        
        mixblock = torch.cat(mix_list, 0)
        mixlabel = torch.cat(mix_label_list, 0)
        
        self.mix_spec = mixblock
        self.mix_label = mixlabel
                
        
    def __len__(self):
        return self.mix_spec.shape[0]

                
    def __getitem__(self, index): 

        mix_spec = self.spec[index]
        mix_label = self.mix_label[index]
        return mix_spec, mix_label


class featureDataSet(Dataset):
    
    def __init__(self, clean_dir):
        

        with open(clean_dir + 'clean3.json') as f:
            feature_list.append(torch.Tensor(json.load(f)))      
        
        featureblock = torch.cat(clean_list, 0)
        self.featurespec = featureblock
                
        
    def __len__(self):
        return self.featurespec.shape[0]

                
    def __getitem__(self, index): 

        featurespec = self.featurespec[index]
        return featurespec

In [ ]:
mixset = mixDataSet(mix_dir)
featureset = featureDataset(clean_dir)

mixloader = torch.utils.data.DataLoader(dataset = mixset,
                                                batch_size = bs,
                                                shuffle = False)

featureloader = torch.utils.data.DataLoader(dataset = featureset,
                                           batch_size = bs,
                                           shuffle = False)

In [ ]:
'''featureNet'''
class featureNet(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1025*16, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 10)

        
    def forward(self, x):
        x = x.view(-1, 1025*16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
#        x = torch.sigmoid(self.fc3(x))
        
        return x

#feature_model = featureNet()
feature_model = torch.load(root_dir + 'FeatureNet.pkl')
    
'''ANet'''
class ANet(nn.Module):
    def __init__(self):
        super(ANet, self).__init__()

        self.linear7 = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
        )
        self.linear6 = nn.Sequential(
            nn.Linear(256, 256),
            nn.ReLU(),
        )
        self.linear5 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
        )
        self.linear4 = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
        )
        self.linear3 = nn.Sequential(
            nn.Linear(256, 32),
            nn.ReLU(),
        )
        self.linear2 = nn.Sequential(
            nn.Linear(256, 16),
            nn.ReLU(),
        )

    def forward(self, x):
        x = x.view(bs, 1, 256)

        a7 = self.linear7(x).view(bs, 512, 1, 1)
        a6 = self.linear6(x).view(bs, 256, 1, 1)
        a5 = self.linear5(x).view(bs, 128, 1, 1)
        a4 = self.linear4(x).view(bs, 64, 1, 1)
        a3 = self.linear3(x).view(bs, 32, 1, 1)
        a2 = self.linear2(x).view(bs, 16, 1, 1)

return a7, a6, a5, a4, a3, a2

A_model = ANet()


''' ResBlock '''
class ResBlock(nn.Module):
    def __init__(self, channels_in, channels_out):
        super(ResBlock, self).__init__()

        self.channels_in = channels_in
        self.channels_out = channels_out

        self.conv1 = nn.Conv2d(in_channels=channels_in, out_channels=channels_out, kernel_size=(3,3), padding=1)
        self.conv2 = nn.Conv2d(in_channels=channels_out, out_channels=channels_out, kernel_size=(3,3), padding=1)

    def forward(self, x):
        if self.channels_out > self.channels_in:
            x1 = F.relu(self.conv1(x), inplace = True)
            x1 =        self.conv2(x1)
            x  = self.sizematch(self.channels_in, self.channels_out, x)
            return F.relu(x + x1, inplace = True)
        elif self.channels_out < self.channels_in:
            x = F.relu(self.conv1(x))
            x1 =       self.conv2(x)
            x = x + x1
            return F.relu(x, inplace = True)
        else:
            x1 = F.relu(self.conv1(x), inplace = True)
            x1 =        self.conv2(x1)
            x = x + x1
            return F.relu(x, inplace = True)

    def sizematch(self, channels_in, channels_out, x):
        zeros = torch.zeros( (x.size()[0], channels_out - channels_in, x.shape[2], x.shape[3]), dtype = torch.float32)
        return torch.cat((x, zeros), dim=1)

class ResTranspose(nn.Module):
    def __init__(self, channels_in, channels_out):
        super(ResTranspose, self).__init__()

        self.channels_in = channels_in
        self.channels_out = channels_out

        self.deconv1 = nn.ConvTranspose2d(in_channels=channels_in, out_channels=channels_out, kernel_size=(2,2), stride=2)
        self.deconv2 = nn.Conv2d(in_channels=channels_out, out_channels=channels_out, kernel_size=(3,3), padding=1)

    def forward(self, x):
        # cin = cout
        x1 = F.relu(self.deconv1(x), inplace = True)
        x1 =        self.deconv2(x1)
        x = self.sizematch(x)
        return F.relu(x + x1, inplace = True)

    def sizematch(self, x):
        # expand
        x2 = torch.zeros(x.shape[0], self.channels_in, x.shape[2]*2, x.shape[3]*2)

        row_x  = torch.zeros(x.shape[0], self.channels_in, x.shape[2], 2*x.shape[3])
        row_x[:,:,:,odd(x.shape[3]*2)]   = x
        row_x[:,:,:,even(x.shape[3]*2)]  = x
        x2[:,:, odd(x.shape[2]*2),:] = row_x
        x2[:,:,even(x.shape[2]*2),:] = row_x

        return x2


def initialize(m):
    if isinstance(m, nn.Conv2d):
        init.xavier_normal_(m.weight)
        init.constant_(m.bias, 0)
    if isinstance(m, nn.ConvTranspose2d):
        init.xavier_normal_(m.weight)



class ResDAE(nn.Module):
    def __init__(self):
        super(ResDAE, self).__init__()

        # 128x128x1

        self.upward_net1 = nn.Sequential(
            ResBlock(1, 1),
            ResBlock(1, 8),
            ResBlock(8, 8),
            nn.BatchNorm2d(8),
        )

        # 64x64x8

        self.upward_net2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=8, kernel_size=(2,2), stride=2),
            nn.ReLU(),
            ResBlock(8, 8),
            ResBlock(8, 16),
            ResBlock(16, 16),
            nn.BatchNorm2d(16),
        )

        # 32x32x16

        self.upward_net3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(2,2), stride=2),
            nn.ReLU(),
            ResBlock(16, 16),
            ResBlock(16, 32),
            ResBlock(32, 32),
            nn.BatchNorm2d(32),
        )

        # 16x16x32

        self.upward_net4 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(2,2), stride=2),
            nn.ReLU(),
            ResBlock(32, 32),
            ResBlock(32, 64),
            ResBlock(64, 64),
            nn.BatchNorm2d(64),
        )

        # 8x8x64

        self.upward_net5 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(2,2), stride=2),
            nn.ReLU(),
            ResBlock(64, 64),
            ResBlock(64, 128),
            ResBlock(128, 128),
            nn.BatchNorm2d(128),
        )

        # 4x4x128

        self.upward_net6 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(2,2), stride=2),
            nn.ReLU(),
            ResBlock(128, 128),
            ResBlock(128, 256),
            ResBlock(256, 256),
            nn.BatchNorm2d(256),
        )

        # 2x2x256

        self.upward_net7 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(2,2), stride=2),
            nn.ReLU(),
            ResBlock(256, 256),
            ResBlock(256, 512),
            ResBlock(512, 512),
            nn.BatchNorm2d(512),
        )

        # 1x1x512
        self.downward_net7 = nn.Sequential(
            ResBlock(512, 512),
            ResBlock(512, 256),
            ResBlock(256, 256),
            ResTranspose(256, 256),
#            nn.ConvTranspose2d(256, 256, kernel_size = (2,2), stride = 2),
            nn.BatchNorm2d(256),
        )

        # 2x2x256

        self.downward_net6 = nn.Sequential(
            # 8x8x64
            ResBlock(256, 256),
            ResBlock(256, 128),
            ResBlock(128, 128),
            ResTranspose(128, 128),
#            nn.ConvTranspose2d(128, 128, kernel_size = (2,2), stride = 2),
            nn.BatchNorm2d(128),
        )

        # 4x4x128
        # cat -> 4x4x256
        self.uconv5 = nn.Conv2d(256, 128, kernel_size=(3,3), padding=(1,1))
        # 4x4x128
        self.downward_net5 = nn.Sequential(
            ResBlock(128, 128),
            ResBlock(128, 64),
            ResBlock(64, 64),
            ResTranspose(64, 64),
#            nn.ConvTranspose2d(64, 64, kernel_size = (2,2), stride = 2),
            nn.BatchNorm2d(64),
        )

        # 8x8x64
        # cat -> 8x8x128
        self.uconv4 = nn.Conv2d(128, 64, kernel_size=(3,3), padding=(1,1))
        # 8x8x64
        self.downward_net4 = nn.Sequential(
            ResBlock(64, 64),
            ResBlock(64, 32),
            ResBlock(32, 32),
            ResTranspose(32, 32),
#            nn.ConvTranspose2d(32, 32, kernel_size = (2,2), stride = 2),
            nn.BatchNorm2d(32),
        )

        # 16x16x32
        # cat -> 16x16x64
        self.uconv3 = nn.Conv2d(64, 32, kernel_size=(3,3), padding=(1,1))
        # 16x16x32
        self.downward_net3 = nn.Sequential(
            ResBlock(32, 32),
            ResBlock(32, 16),
            ResBlock(16, 16),
            ResTranspose(16, 16),
#            nn.ConvTranspose2d(16, 16, kernel_size = (2,2), stride = 2),
            nn.BatchNorm2d(16),
        )

        # 32x32x16
        # cat -> 32x32x32
        self.uconv2 = nn.Conv2d(32, 16, kernel_size=(3,3), padding=(1,1))
        # 32x32x16
        self.downward_net2 = nn.Sequential(
            ResBlock(16, 16),
            ResBlock(16, 8),
            ResBlock(8, 8),
            ResTranspose(8, 8),
#            nn.ConvTranspose2d(8, 8, kernel_size = (2,2), stride = 2),
            nn.BatchNorm2d(8),
        )

        # 64x64x8
        self.downward_net1 = nn.Sequential(
            ResBlock(8, 8),
            ResBlock(8, 1),
            ResBlock(1, 1),
            ResBlock(1, 1),
            nn.BatchNorm2d(1),
        )

        # 128x128x1
        
        self.apply(initialize)


    def upward(self, x, a7= True, a6= True, a5= True, a4= True, a3= True, a2= True):
        x = x.view(bs, 1, 256, 128)

        # 1x256x128
#        print ("initial", x.shape)

        x = self.upward_net1(x)
#        print ("after conv1", x.shape)


        # 8x128x64
        x = self.upward_net2(x)
        if a2 is not None: x = x * a2
        self.x2 = x
#        print ("after conv2", x.shape)

        # 16x64x32
        x = self.upward_net3(x)
        if a3 is not None: x = x * a3
        self.x3 = x
#        print ("after conv3", x.shape)

        # 32x32x16
        x = self.upward_net4(x)
        if a4 is not None: x = x * a4
        self.x4 = x
#        print ("after conv4", x.shape)

        # 64x16x8
        x = self.upward_net5(x)
        if a5 is not None: x = x * a5
        self.x5 = x
#        print ("after conv5", x.shape)

        
        # 128x8x4
        x = self.upward_net6(x)
        if a6 is not None: x = x * a6
#        print ("after conv6", x.shape)

        # 256x4x2
        x = self.upward_net7(x)
        if a7 is not None: x = x * a7
#        print ("after conv7", x.shape)

        # 512x2x1
        return x


    def downward(self, y, shortcut= True):
#        print ("begin to downward, y.shape = ", y.shape)
        # 512x2x1
        y = self.downward_net7(y)
#        print ("after down7", y.shape)

        # 256x4x2
        y = self.downward_net6(y)
#        print ("after down6", y.shape)

        # 128x8x4
        if shortcut:
            y = torch.cat((y, self.x5), 1)
            y = F.relu(self.uconv5(y))
        y = self.downward_net5(y)
#        print ("after down5", y.shape)

        # 64x16x8
        if shortcut:
            y = torch.cat((y, self.x4), 1)
            y = F.relu(self.uconv4(y))
        y = self.downward_net4(y)
#        print ("after down4", y.shape)

        # 32x32x16
        if shortcut:
            y = torch.cat((y, self.x3), 1)
            y = F.relu(self.uconv3(y))
        y = self.downward_net3(y)
#        print ("after down3", y.shape)

        # 16x64x32
        if shortcut:
            y = torch.cat((y, self.x2), 1)
            y = F.relu(self.uconv2(y))
        y = self.downward_net2(y)
#        print ("after down2", y.shape)

        # 8x128x64
        y = self.downward_net1(y)
#        print ("after down1", y.shape)
 
        # 1x256x128
        return y

# Res_model = ResDAE()
Res_model = torch.load(root_dir + 'recover/SSIM-CONV/DAE_SSIM.pkl')
# print (model)

In [ ]:
#import pytorch_ssim
criterion = pytorch_ssim.SSIM()
optimizer = torch.optim.Adam(model.parameters(), lr = lr) #, momentum = mom)

In [ ]:
loss_record = []

In [ ]:
model.train()
for epo in range(epoch):
    for i, data in enumerate(trainloader, 0):
        inputs = data
        inputs = Variable(inputs)
        optimizer.zero_grad()
        top = model.upward(inputs + white(inputs))
        
        outputs = model.downward(top, shortcut = True)
        inputs = inputs.view(bs, 1, 256, 128)
        outputs = outputs.view(bs, 1, 256, 128)
        #with open ( root_dir + 'recover/L1loss_FC/recover_pic_epo_' + str(epo), 'w') as f:
        #    json.dump(outputs.tolist(), f)
        
        loss = - criterion(outputs, inputs)
        ssim_value = - loss.data.item()
        loss.backward()
        optimizer.step()
        
        loss_record.append(loss.item())
        plt.figure(figsize = (20, 10))
        plt.plot(loss_record)
        plt.xlabel('iterations')
        plt.ylabel('loss')
        plt.savefig(root_dir + 'recover/SSIM-CONV/DAE_loss.png')
        
        if i % 20 == 0:
            inn = inputs[0].view(256, 128).detach().numpy() * 255
            cv2.imwrite("/home/tk/Documents/recover/SSIM-CONV/" + str(epo) + "_" + str(i) + ".png", inn)
            
            out = outputs[0].view(256, 128).detach().numpy() * 255
            cv2.imwrite("/home/tk/Documents/recover/SSIM-CONV/" + str(epo) + "_" + str(i) + "_re.png", out)
            
            
#        every_loss.append(loss.item())
#        del inputs, data

        if i % 10 == 0:
            print ('[%d, %5d] loss: %.3f' % (epo, i, loss.item()))
#            print ('[%d, %5d] ssim: %.3f' % (epo, i, ssim_value))
   
    gc.collect()
    plt.close("all")

#    epoch_loss.append(np.mean(every_loss))
#    every_loss = []
    


#        plt.figure(figsize = (20, 10))
#        plt.plot(epoch_loss)
#        plt.xlabel('epocs')
#        plt.ylabel('epoch_loss')
#        plt.savefig(root_dir + 'DAE_epoch_loss')

    
#=============================================
#        Save Model & Loss
#=============================================

torch.save(model, root_dir + 'recover/SSIM-CONV/DAE_SSIM.pkl')

In [ ]:
import torch
import torch.DataSets

In [ ]:
mix_list = []
mix_label_list = []
class mixDataSet(Dataset):
    def __init__(self, mix_dir, mix_label_dir):           
        
        with open(mix_dir + 'mix3.json') as f:
            mix_list.append(torch.Tensor(json.load(f)))
        
        with open(mix_label_dir + 'mix_label3.json') as f:
            mix_label_list.append(torch.Tensor(json.load(f)))
        
        mixblock = torch.cat(mix_list, 0)
        mixlabel = torch.cat(mix_label_list, 0)
        
        self.mix_spec = mixblock
        self.mix_label = mixlabel
        
    def __len__(self):
        return self.mix_spec.shape[0]

    def __getitem__(self, spec_index, label_index): 

        mix_spec = self.mix_spec[spec_index]
        mix_label = self.mix_label[label_index]
        return mix_spec, mix_label

In [ ]:
mixset = mixDataSet(mix_dir, mix_label_dir)

In [ ]:
mixset.__len__()

In [ ]:
mixset.__getitem__(150,19)

In [ ]:
jh = open('/home/tk/Documents/mix/mix3.json').read()
a = json.loads(jh)

In [ ]:
jh = open('/home/tk/Documents/mix_labels/mix_label3.json').read()
b = json.loads(jh)

In [ ]:
a = np.array(a)
b = np.array(b)

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
b = np.array(b)
b.shape

In [ ]:
b[]

In [ ]:
import numpy as np
a = np.array(a)

In [ ]:
a.shape

In [ ]:
a[140]

In [ ]:
mixfolder = os.listdir(mix_dir)
mixfolder.sort()

In [ ]:
mixfolder

In [ ]:
clean_dir = root_dir + 'clean/' 
mix_dir = root_dir + 'mix/' 
clean_label_dir = root_dir + 'clean_labels/' 
mix_label_dir = root_dir + 'mix_labels/' 

In [ ]:
a = [1.0623e-07, 1.3231e-03, 5.5651e-06, 2.8495e-02, 1.3299e-04, 5.2328e-06, 7.2000e-04, 2.8051e-05, 3.9795e-02, 9.8838e-01]

In [ ]:
torch.max(a, 0)

In [ ]:
a = torch.tensor([[1.0578e-02, 3.3309e-01, 5.5529e-01, 1.0909e-02, 3.8428e-02, 1.3777e-03,
         1.2060e-02, 4.5675e-02, 1.5102e-01, 5.5134e-01],
        [4.0438e-05, 2.1043e-04, 1.2589e-01, 5.1516e-03, 6.8046e-04, 9.9534e-03,
         1.8230e-03, 5.8381e-06, 2.6256e-01, 7.9211e-01],
        [1.2880e-03, 2.0016e-02, 3.5143e-02, 4.8582e-02, 1.2939e-02, 1.1453e-02,
         1.3557e-01, 3.3008e-02, 2.6488e-01, 2.6349e-01],
        [3.3836e-03, 7.3263e-02, 1.7925e-02, 2.2843e-01, 4.1886e-02, 1.5798e-02,
         2.4491e-02, 5.4132e-03, 1.6431e-01, 2.7407e-02],
        [5.5855e-02, 1.1084e-01, 1.7244e-02, 6.8025e-04, 2.2809e-02, 3.1570e-01,
         3.4346e-03, 3.9227e-01, 7.6018e-03, 1.7340e-01],
        [2.1523e-01, 7.6109e-02, 4.9581e-02, 9.8011e-04, 1.0011e-01, 2.1144e-01,
         1.9372e-02, 4.0609e-02, 1.3001e-02, 2.5672e-03],
        [7.3727e-02, 2.0658e-01, 9.7838e-02, 5.0830e-04, 4.4293e-03, 4.8621e-03,
         6.2715e-02, 4.8715e-02, 2.3443e-03, 8.7680e-04],
        [1.0301e-04, 2.3858e-03, 1.3977e-02, 6.8922e-04, 8.5931e-03, 2.2245e-04,
         9.8420e-01, 3.0137e-03, 6.7784e-02, 6.7894e-03],
        [2.2825e-03, 1.6498e-01, 2.2518e-02, 1.0465e-03, 7.4058e-03, 8.8901e-01,
         8.0338e-02, 1.3140e-01, 1.8809e-03, 3.2917e-03],
        [8.2398e-03, 5.5224e-02, 2.2561e-02, 1.9494e-04, 2.3196e-02, 3.1043e-03,
         3.8204e-01, 9.6084e-03, 1.9456e-03, 1.7574e-03],
        [1.0623e-07, 1.3231e-03, 5.5651e-06, 2.8495e-02, 1.3299e-04, 5.2328e-06,
         7.2000e-04, 2.8051e-05, 3.9795e-02, 9.8838e-01],
        [3.3474e-03, 1.8030e-03, 9.8147e-01, 8.5557e-05, 2.4013e-05, 1.0115e-02,
         8.4762e-04, 1.3052e-03, 7.6764e-03, 1.2625e-04],
        [2.0284e-03, 6.2676e-04, 1.0758e-04, 1.1942e-06, 1.7032e-05, 2.7349e-04,
         5.7749e-03, 2.9111e-01, 6.8634e-04, 1.4915e-04],
        [1.3000e-03, 2.7204e-05, 2.6371e-03, 4.6776e-04, 1.0860e-05, 8.1055e-04,
         3.0986e-04, 2.8704e-06, 9.5868e-02, 1.2196e-01],
        [8.1448e-04, 5.3910e-06, 4.3349e-05, 9.9108e-01, 2.1359e-05, 2.3141e-08,
         3.6492e-02, 8.2024e-09, 1.9351e-01, 6.2107e-05],
        [4.6230e-05, 3.4710e-02, 2.7775e-03, 1.2075e-05, 1.6022e-04, 9.7403e-01,
         1.1194e-02, 5.7854e-02, 4.9520e-06, 6.2279e-06]])


In [ ]:
_, predict = torch.max(a, 1)

In [ ]:
predict

In [ ]:
blank = []
for i in predict:
    b = np.zeros(10)
    b[i] = 1
    blank.append(b)
    

In [ ]:
b

In [ ]:
print (blank)

In [ ]:
labels = torch.tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [ ]:
predicted = torch.tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=torch.float64)

In [ ]:
labels == predicted

In [ ]:
9/16

In [ ]:
class featureDataSet(Dataset):
    def __init__(self, clean_dir, clean_label_dir):
                
        for i in cleanfolder:
            with open(clean_dir + '{}'.format(i)) as f:
                clean_list.append(torch.Tensor(json.load(f)))
                
        for i in cleanlabelfolder:
            with open(clean_label_dir + '{}'.format(i)) as f:
                clean_label_list.append(torch.Tensor(json.load(f)))
        
        cleanblock = torch.cat(clean_list, 0)
        self.spec = torch.cat([cleanblock], 0)
                
        cleanlabel = torch.cat(clean_label_list, 0)
        self.label = torch.cat([cleanlabel], 0)

        
    def __len__(self):
        return self.spec.shape[0]

                
    def __getitem__(self, index): 

        spec = self.spec[index]
        label = self.label[index]
        return spec, label

In [ ]:
root_dir = '/home/tk/Documents/'
clean_dir = root_dir + 'clean/' 
mix_dir = root_dir + 'mix/' 
clean_label_dir = root_dir + 'clean_labels/' 
mix_label_dir = root_dir + 'mix_labels/' 

cleanfolder = os.listdir(clean_dir)
cleanfolder.sort()

cleanlabelfolder = os.listdir(clean_label_dir)
cleanlabelfolder.sort()

clean_list = []
clean_label_list = []

In [ ]:
c = featureDataSet(clean_dir, clean_label_dir)
spec, label = c.__getitem__(0)

In [ ]:
print (spec, label)

#### White noise

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

def white_noise(num_samples):
    mean = -10
    std = 1
    num_samples = num_samples
    samples = np.random.normal(mean, std, size = num_samples)

    return samples

#### Pink Noise

In [ ]:
def voss(nrows, ncols = 16):
    
    import pandas as pd
    array = np.empty((nrows, ncols))
    array.fill(np.nan)
    array[0, :] = np.random.random(ncols)
    array[:, 0] = np.random.random(nrows)
    
    n = nrows
    cols = np.random.geometric(0.5, n)
    cols[cols >= ncols] = 0
    
    rows = np.random.randint(nrows, size = n)
    array[rows, cols] = np.random.random(n)
    
    df = pd.DataFrame(array)
    df.fillna(method = 'ffill', axis = 0, inplace = True)
    total = df.sum(axis = 1)
    
    return total.values

In [ ]:
y = voss(1000, 16)

In [ ]:
plt.plot(y)
plt.show()

In [ ]:
import colorednoise

beta = 1
samples = 1000
y = colorednoise.powerlaw_psd_gaussian(beta, samples)
plt.plot(y)
plt.show()

In [ ]:
len(y)

In [ ]:
y = np.array(y).reshape(100, 10)
y.shape

In [ ]:
import cv2

In [ ]:
j = open('/home/tk/Documents/clean/clean10.json').read()
a = json.loads(j)
a = np.array(a)
print(a.shape)

In [ ]:
a[1]
cv2.imwrite('/home/tk/Desktop/original.png', a[1]*255)

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

def white_noise(num_samples):
    mean = 0
    std = 0.3
    num_samples = num_samples
    samples = np.random.normal(mean, std, size = num_samples)

    return samples

In [ ]:
white = white_noise(256*128)
white = white.reshape(256, 128)

a_white = a[1] + white
cv2.imwrite('/home/tk/Desktop/original_white.png', a_white * 255)

In [ ]:
a_white = a[1] + white
cv2.imwrite('/home/tk/Desktop/original_white.png', a_white * 255)

In [ ]:
def slice_it(filename, input_path, output_path, length):
    from pydub import AudioSegment
    from pydub.utils import make_chunks
    

    # slice the file
    myaudio = AudioSegment.from_file(input_path + filename, 'wav') 
    chunks = make_chunks(myaudio, length) # Make chunks

    #Export all individual chunks as wav files
    for i, chunk in enumerate(chunks):
        chunk_name = filename[:-4] + "_{0}.wav".format(i) # select first 6 characters as file name
        print ("exporting", chunk_name)
        chunk.export(output_path + chunk_name, format="wav")

    # dump the last slice (might be an incomplete slice)
    dump_file = [] 
    chunk_list = os.listdir(output_path)
    if '.DS_Store' in chunk_list:
        chunk_list.remove('.DS_Store')

In [ ]:
len(sliced_list)

In [ ]:
full_audio = ['birdstudybook_', 'captaincook_', 'cloudstudies_02_clayden_12_', 
              'constructivebeekeeping_',
              'discoursesbiologicalgeological_16_huxley_12_', 
              'natureguide_', 'pioneersoftheoldsouth_', 
              'pioneerworkalps_02_harper_12_', 
              'romancecommonplace_', 'travelstoriesretold_']

In [ ]:
len(sliced_list)

In [ ]:
sliced_list[700]

In [ ]:
cnt = 0
sliced_list = []
for name in full_audio:
    for i in range(60, 160):
        m = name + str(i) + '.wav'
        sliced_list.append(m)
        del m
        cnt +=1
        
print (sliced_list)
print (cnt)

In [ ]:
import os
for j in sliced_list:
    slice_it(j, '/home/tk/Documents/slice_10sec/', '/home/tk/Documents/sliced_pool/', 500)

In [ ]:
p = os.listdir('/home/tk/Documents/slice_pointsec/')

for ev in p:
    os.remove('/home/tk/Documents/slice_pointsec/' + ev)

In [ ]:
g = os.listdir('/home/tk/Documents/sliced_pool/')
print (len(g))

In [ ]:
selected_audio = np.random.choice(g, size = 2, replace = False)
if selected_audio[0][:8] == selected_audio[1][:8]:
    selected_audio = np.random.choice(g, size = 2, replace = False)
if r[0][:8] == r[1][:8]:
    selected_audio = np.random.choice(g, size = 2, replace = False)
print (selected_audio)

In [ ]:
import cv2
cv2.imwrite('/home/tk/Desktop/spec0.png', spec0 * 255)
cv2.imwrite('/home/tk/Desktop/spec1.png', spec1 * 255)
cv2.imwrite('/home/tk/Desktop/mixed_spec.png', mixed_spec * 255)


In [ ]:
root_dir = '/home/tk/Documents/'

stock = []
spec0_cluster = []
spec1_cluster = []

for j in range(10):
    for i in range(100):
        from data_process import gen_spectrogram

        selected_audio = np.random.choice(g, size = 2, replace = False)

        # prevent to select from the same source
        if selected_audio[0][:8] == selected_audio[1][:8]: 
            selected_audio = np.random.choice(g, size = 2, replace = False)
        if r[0][:8] == r[1][:8]:
            selected_audio = np.random.choice(g, size = 2, replace = False)
        print (selected_audio)

        # generate 2 spectrograms & mix
        gen_path0 = '/home/tk/Documents/sliced_pool/' + selected_audio[0]
        gen_path1 = '/home/tk/Documents/sliced_pool/' + selected_audio[1]
        spec0 = gen_spectrogram(gen_path0)
        spec1 = gen_spectrogram(gen_path1)
        mixed_spec = spec0 + spec1

        # append the mixed spectrograms
        stock.append(mixed_spec)
        spec0_cluster.append(spec0)
        spec1_cluster.append(spec1)

    stock = np.array(stock)
    print ('mixed_spec shape = ', stock.shape)

    spec0_cluster = np.array(spec0_cluster)
    print ('spec0 cluster shape = ', spec0_cluster.shape)

    spec1_cluster = np.array(spec1_cluster)
    print ('spec1 cluster shape = ', spec1_cluster.shape)

    with open(root_dir + "mix_pool/mix_spec/" + str(j) + '.json', 'w') as jh:
        json.dump(stock.tolist(), jh)

    with open(root_dir + "mix_pool/spec0/" + str(j) + '.json', 'w') as jh:
        json.dump(spec0_cluster.tolist(), jh)

    with open(root_dir + "mix_pool/spec1/" + str(j) + '.json', 'w') as jh:
        json.dump(spec1_cluster.tolist(), jh)

In [ ]:
stock = np.array(stock)

In [ ]:
stock.shape

In [ ]:
import random
random.seed(7)

In [ ]:
from pydub import AudioSegment
from pydub.utils import make_chunks
import os
import gc
import matplotlib.pyplot as plt
import scipy
import scipy.io.wavfile
import numpy as np
import json
import data_process

In [ ]:
full_audio = ['birdstudybook_', 'captaincook_', 'cloudstudies_02_clayden_12_', 
              'constructivebeekeeping_',
              'discoursesbiologicalgeological_16_huxley_12_', 
              'natureguide_', 'pioneersoftheoldsouth_', 
              'pioneerworkalps_02_harper_12_', 
              'romancecommonplace_', 'travelstoriesretold_']

folder_name = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

seq = enumerate(full_audio, folder_name)


In [ ]:
from pydub import AudioSegment
from pydub.utils import make_chunks
import os
import gc
import matplotlib.pyplot as plt
import scipy
import scipy.io.wavfile
import numpy as np
import json
import data_process

# parameters:

## 10 sec slicing:
full_audio_path = '/home/tk/Documents/full_audio/' # full audio will be stored here
sec10_sliced_path = '/home/tk/Documents/slice_10sec/' # 10 sec sliced will be stored here

In [ ]:
a = [1,2,3,4,5]
a = np.mean(a)
print (a)

In [ ]:
a = np.random.randn(200, 200)
b = np.random.randn(200, 200)

In [ ]:
c = np.zeros((200,200))
c = c + a+ b

In [ ]:
import numpy as np
import os
from data_process import gen_spectrogram
import json

root_dir = '/home/tk/Documents/'
sliced_pool_path = '/home/tk/Documents/sliced_pool/'
mixed_pool_path =  '/home/tk/Documents/mix_pool/'

full_audio = ['birdstudybook', 'captaincook', 'cloudstudies_02_clayden_12', 
              'constructivebeekeeping',
              'discoursesbiologicalgeological_16_huxley_12', 
              'natureguide', 'pioneersoftheoldsouth', 
              'pioneerworkalps_02_harper_12', 
              'romancecommonplace', 'travelstoriesretold']

In [ ]:
blocks = 10
for i in range(blocks):
    for name in full_audio:
        
        all_clean_spec = []
        if (mixed_pool_path + 'feature/' + name) == False:
            os.mkdir(mixed_pool_path + 'feature/' + name)
        
        file_name_list = os.listdir(sliced_pool_path + name + '/clean/')
        file_name = np.random.choice(file_name_list, 100)
        
        for k in file_name:
            spec = gen_spectrogram(sliced_pool_path + name + '/clean/' + k)
            print (k)
            all_clean_spec.append(spec)
            
            
        all_clean_spec = np.array(all_clean_spec)
        all_clean_spec = np.stack(all_clean_spec)
            
        print ("name = ", name , ", shape = ", all_clean_spec.shape)
       
    with open(mixed_pool_path +  '/feature/' + name + '/' + str(i) + '.json', 'w') as jh:
        json.dump(all_clean_spec.tolist(), jh)




In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data
import torch.nn.init as init
import pytorch_ssim

import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import pickle
import os
import json
import numpy as np
import gc
import cv2

server = False

root_dir = '/home/tk/Documents/'
if server == True:
    root_dir = '/home/guotingyou/cocktail_phase2/'


clean_dir = '/home/tk/Documents/mix_pool/feature/' 
mix_dir = '/home/tk/Documents/mix_pool/mix_spec/' 
# clean_label_dir = '/home/tk/Documents/clean_labels/' 
mix_label_dir = '/home/tk/Documents/mix_pool/mix_labels/' 
target_spec_dir = '/home/tk/Documents/mix_pool/target_spec/'
target_label_dir = '/home/tk/Documents/mix_pool/target_label/'

In [57]:
class mixDataSet(Dataset):
    
    def __init__(self, mix_dir, target_spec_dir, target_label_dir):           
        
        mix_list = []
        target_spec_list = []
        target_label_list = []

        with open(mix_dir + 'mix_spec2.json') as f:
            mix_list.append(torch.Tensor(json.load(f)))
        
        with open(target_spec_dir + 'target_spec2.json') as f:
            target_spec_list.append(torch.Tensor(json.load(f)))
            
        with open(target_label_dir + 'target_label2.json') as f:
            target_label_list.append(torch.Tensor(json.load(f)))
        
        

        mixblock = torch.cat(mix_list, 0)
        targetspec = torch.cat(target_spec_list, 0)
        targetlabel = torch.cat(target_label_list, 0)
        
        self.mix_spec = mixblock
        self.target_spec = targetspec
        self.target_label = targetlabel

    def __len__(self):
        return self.mix_spec.shape[0]


    def __getitem__(self, index): 

        mix_spec = self.mix_spec[index]
        target_spec = self.target_spec[index]
        target_label = self.target_label[index]

        return mix_spec, target_spec, target_label


class featureDataSet(Dataset):
    
    def __init__(self, clean_dir, label):
        
        full_audio = ['birdstudybook', 'captaincook', 'cloudstudies_02_clayden_12', 
              'constructivebeekeeping',
              'discoursesbiologicalgeological_16_huxley_12', 
              'natureguide', 'pioneersoftheoldsouth', 
              'pioneerworkalps_02_harper_12', 
              'romancecommonplace', 'travelstoriesretold']

        feature_list = []

        with open(clean_dir + full_audio[label] + '/0.json') as f:
            featureblock = torch.Tensor(json.load(f))     
        
#        featureblock = torch.cat(feature_list, 0)
        
        self.featurespec = featureblock
                
        
    def __len__(self):
        return self.featurespec.shape[0]

                
    def __getitem__(self): 
        
        featurespec = self.featurespec
        return featurespec

In [63]:
with open(clean_dir + 'birdstudybook/0.json') as f:
    featureblock = torch.Tensor(json.load(f))     

featureblock.shape

torch.Size([10, 256, 128])

In [66]:
b = torch.mean(featureblock, 0, True)
b.shape

torch.Size([1, 256, 128])

In [58]:
bs = 1

In [40]:
#=============================================
#        Define Dataloader
#=============================================


mixset = mixDataSet(mix_dir, target_spec_dir, target_label_dir)

mixloader = torch.utils.data.DataLoader(dataset = mixset,
                                        batch_size = bs,
                                        shuffle = False)

In [41]:
m, t, l = mixset.__getitem__(3)

In [42]:
l

tensor(5.)

In [46]:
l = torch.tensor([1., 6., 5., 5.])

In [44]:
int(l[0])

1

In [59]:
featureset = featureDataSet(clean_dir, 2)
a = featureset.__getitem__()

In [60]:
a.shape

torch.Size([10, 256, 128])